## Vectorisation des textes (text_for_embedding) et stockage dans une base de données vectorielle ##


Je commence par vectoriser les textes en vecteurs mathématiques en utilisant le modèle multilingue SentenceTransformers de Hugging Face : paraphrase-multilingual-MiniLM-L12-v2

In [188]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import uuid
import json

# --- CONFIGURATION FRANÇAISE 🇫🇷 ---
EMBEDDING_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
COLLECTION_NAME = "royaume_du_muffin_6"
FICHIER_JSON = "base_de_donnees.json"

def load_and_simulate_data():
    
    with open(FICHIER_JSON, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return pd.DataFrame(data)


def create_embeddings_and_store(df):
    print("🤖 Chargement du modèle d'embedding multilingue...")
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)

    df_copy = df.copy()
    df_copy = df_copy.fillna("")
    
    # On transforme la colonne 'ingredients' (liste) en texte (str)
    # On vérifie d'abord si c'est bien une liste pour éviter de faire planter le code
    df_copy['ingredients'] = df_copy['ingredients'].apply(
        lambda x: ", ".join(x) if isinstance(x, list) else x
    )
    # Concaténation Titre + Ingrédients pour la recherche
    documents = df_copy["text_for_embedding"].tolist()
    metadatas = df_copy.to_dict(orient='records')
    ids = [str(uuid.uuid4()) for _ in range(len(df_copy))]

    print("⚡ Vectorisation en cours...")
    embeddings = model.encode(documents).tolist()


    # Stockage ChromaDB
    client = chromadb.Client() # En mémoire pour le test
    try: client.delete_collection(name=COLLECTION_NAME)
    except: pass

    collection = client.create_collection(name=COLLECTION_NAME)
    collection.add(documents=documents, embeddings=embeddings, metadatas=metadatas, ids=ids)

    print(f"✅ Indexation terminée ! {collection.count()} recettes stockées.")
    return collection



In [189]:
# --- TEST ---
if __name__ == "__main__":
    df = load_and_simulate_data()
    db = create_embeddings_and_store(df)

🤖 Chargement du modèle d'embedding multilingue...
⚡ Vectorisation en cours...
✅ Indexation terminée ! 375 recettes stockées.


In [203]:
# Test de recherche en français
query = "Je suis fan de pizza"
model = SentenceTransformer(EMBEDDING_MODEL_NAME)
results = db.query(query_embeddings=model.encode([query]).tolist(), n_results=6)
print(f"\n🔎 Question: '{query}'")

for i, metadata in enumerate(results['metadatas'][0]):
    print(f"👉 Top {i+1}: {metadata['titre']}")


🔎 Question: 'Je suis fan de pizza'
👉 Top 1: Pizza façon muffin
👉 Top 2: Muffins pizzas faciles
👉 Top 3: Muffins 'pizza'
👉 Top 4: Muffins burger à l'italienne
👉 Top 5: Muffin feuilleté tomate aubergine mozzarella
👉 Top 6: Muffins au fromage à raclette
